In [1]:
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### Collect tweets with hashtag `#30mapchallenges`

In [2]:
# Define the search term and the date_since date as variables
search_words = "#30DayMapChallenge -filter:retweets"
since = "2020-10-30"

In [3]:
# Collect tweets
tweets = tweepy.Cursor(api.search,
              q=search_words,
              since= since, include_entities=True).items()

In [4]:
for tweet in tweets:
    print(tweet)
    break

Status(_api=<tweepy.api.API object at 0x000001B9AEE27148>, _json={'created_at': 'Tue Nov 03 09:36:29 +0000 2020', 'id': 1323559667684040704, 'id_str': '1323559667684040704', 'text': "#30DayMapChallenge\n#Day3 #Polygons - Topical map for today's election or why counties are a bad base unit for anyth… https://t.co/dr4W6rZibD", 'truncated': True, 'entities': {'hashtags': [{'text': '30DayMapChallenge', 'indices': [0, 18]}, {'text': 'Day3', 'indices': [19, 24]}, {'text': 'Polygons', 'indices': [25, 34]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/dr4W6rZibD', 'expanded_url': 'https://twitter.com/i/web/status/1323559667684040704', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to

In [51]:
def collect_tweets(key_word):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,q=key_word, count=1000).items(1000):
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media', []))
        except KeyError:
            pass
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet_count':retweet_count,
                        'favorite_count':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user_screen_name':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [57]:
df = collect_tweets(search_words)

df = df[(df.tweet_created_at >= '2020-11-02 00:00:00') & 
   (df.tweet_created_at <= '2020-11-03 00:00:00')]

df.to_csv(f"./data/2020-11-02.csv")

## Top retweeted `#30mapchallenges` post

In [58]:
df.sort_values(by='favorite_count', ascending=False).head(15)

,tweet_id,tweet_text,language,tweet_created_at,retweet_count,favorite_count,hashtags,user_id,user_name,user_screen_name,user_loc,user_follower,friends_count,media
435,1323261174876524546,#30DayMapChallenge \nMap 2: Lines (Only one li...,en,2020-11-02 13:50:23,82,541,"[{'text': '30DayMapChallenge', 'indices': [0, ...",272936200,Raj Bhagat P #Mapper4Life,rajbhagatt,"Nagercoil, Tamil Nadu",17031,1687,[]
271,1323347949389230081,#30DayMapChallenge Day 2: Lines. \n\nMap showi...,en,2020-11-02 19:35:12,17,136,"[{'text': '30DayMapChallenge', 'indices': [0, ...",314881866,Joe Davies,JoeWDavies,Cardiff / Madrid / Luxembourg,152,170,[]
248,1323354734946742272,#30DayMapChallenge (Lines)\nDriving routes &am...,en,2020-11-02 20:02:10,12,120,"[{'text': '30DayMapChallenge', 'indices': [0, ...",3309914684,Sarah Bell,sarahbellmaps,"Bellingham, Washington",5028,401,[]
468,1323242529559556098,A lot of great maps 🗺️ are created during the ...,en,2020-11-02 12:36:18,29,96,"[{'text': '30DayMapChallenge', 'indices': [46,...",2810559039,Jakub Nowosad,jakub_nowosad,,3843,217,[]
272,1323347898797608960,Day 3 of #30DayMapChallenge – Polygons\n\nLow-...,en,2020-11-02 19:35:00,13,89,"[{'text': '30DayMapChallenge', 'indices': [9, ...",1020137956869222401,Andrew Tyrrell,SouthArrowMaps,"Christchurch, New Zealand",1069,187,[]
352,1323302196239368192,#30DayMapChallenge day 2: lines\n\nA simple ma...,en,2020-11-02 16:33:24,9,86,"[{'text': '30DayMapChallenge', 'indices': [0, ...",489547971,Richard Zimerman,RichardZimerman,"Zurich, Switzerland",369,316,[]
625,1323128627085586438,My current estimate is that the #30DayMapChall...,en,2020-11-02 05:03:41,1,86,"[{'text': '30DayMapChallenge', 'indices': [32,...",2198358416,Topi Tjukanov,tjukanov,"Helsinki, Suomi",11667,2515,[]
560,1323189801189842946,Ukrainian cities with a population of more tha...,en,2020-11-02 09:06:46,21,84,[],14992557,gontsa,gontsa,"Cherkasy, Ukraine",4500,1409,[]
545,1323202439755886592,🌍🗺️🦅 Jour 2 du #30DayMapChallenge : les lignes...,fr,2020-11-02 09:57:00,6,80,"[{'text': '30DayMapChallenge', 'indices': [15,...",138790830,Jules Grandin,JulesGrandin,Paris,21625,1006,[]
372,1323292809072308224,#30DayMapChallenge Day 2: lines\n\n#sneakpeek ...,en,2020-11-02 15:56:05,7,71,"[{'text': '30DayMapChallenge', 'indices': [0, ...",20668314,Kenneth Field,kennethfield,"Redlands, CA & global",12214,103,[]


## Top favorited `#30mapchallenges` post

In [11]:
df.sort_values(by='retweet_count', ascending=False).head(15)

,tweet_id,tweet_text,language,tweet_created_at,retweet_count,favorite_count,hashtags,user_id,user_name,user_screen_name,user_loc,user_follower,friends_count,media
895,1322926229096464385,Empezamos el #30DayMapChallenge !!! 🤓🤟\nDia 1:...,es,2020-11-01 15:39:26,262,464,"Santiago, Chile",2380934868,Juanizio Correa,Juanizio_C,"Santiago, Chile",13153,5446,[]
919,1322911629843988480,#30DayMapChallenge \nMap 1: Points\n\nThermal ...,en,2020-11-01 14:41:25,86,342,"Nagercoil, Tamil Nadu",272936200,Raj Bhagat P #Mapper4Life,rajbhagatt,"Nagercoil, Tamil Nadu",17031,1687,[]
849,1322959001517248514,20 years of basking shark sightings 🦈\n\n#30Da...,en,2020-11-01 17:49:40,44,240,London,1101617286,Helen McKenzie,Helenmckmck,London,1264,560,"[{'id': 1322958951567282176, 'id_str': '132295..."
946,1322890973698007044,🌍🔴🥙 Jour 1 du #30DayMapChallenge : les points....,fr,2020-11-01 13:19:20,32,255,Paris,138790830,Jules Grandin,JulesGrandin,Paris,21625,1006,[]
993,1322859560286199809,Scotland in two place-names components: Allt- ...,en,2020-11-01 11:14:31,26,70,,1081900377853829120,D&G place-names,dgplacenames,,542,294,[]
934,1322900054970163200,#30DayMapChallenge Day 1: points\n\nFor the Al...,en,2020-11-01 13:55:26,24,105,"Santiago de Compostela, España",895213223724666880,Dominic Royé,dr_xeo,"Santiago de Compostela, España",2340,986,[]
837,1322965333318094848,#30DayMapChallenge Day 1: points\n\n#sneakpeek...,en,2020-11-01 18:14:49,22,136,"Redlands, CA & global",20668314,Kenneth Field,kennethfield,"Redlands, CA & global",12214,103,[]
888,1322931943021907970,~ points -&gt; lines\n\ncanvas + javascript\n\...,en,2020-11-01 16:02:08,22,203,Italia,2914726517,stefano bovio,allyoucanmap,Italia,636,182,"[{'id': 1322931206787354626, 'id_str': '132293..."
778,1322992130630078464,"Día 1, puntos. #30DayMapChallenge. En CDMX en ...",es,2020-11-01 20:01:18,20,69,"Mexico, DF",36121150,Irving MA,moaimx,"Mexico, DF",3973,4402,[]
990,1322861381272719360,The theme on day 1 of the year's #30DayMapChal...,en,2020-11-01 11:21:45,18,141,"Sheffield, England",343928585,Alasdair Rae,undertheraedar,"Sheffield, England",18569,3409,[]


### Send tweet via API

In [3]:
api.update_status("Look, I'm tweeting from #Python")


Status(_api=<tweepy.api.API object at 0x0000011E60202A48>, _json={'created_at': 'Mon Nov 02 22:53:36 +0000 2020', 'id': 1323397878983856130, 'id_str': '1323397878983856130', 'text': "Look, I'm tweeting from #Python", 'truncated': False, 'entities': {'hashtags': [{'text': 'Python', 'indices': [24, 31]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://hn303.github.io/CamLandEc-RM03/" rel="nofollow">Lonon_REST</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 528087151, 'id_str': '528087151', 'name': 'Haifeng Niu', 'screen_name': 'niu_haifeng', 'location': 'Cambridge, England', 'description': 'PhD candidate in Lab of Interdisciplinary Spatial Analysis, Department of Land Economy, University of Cambridge @CambridgeLandEc', 'url': 'https://t.co/nuaPhizg9Y', 'entities': {'url': {'urls': [{'url': 'https://t.co/nuaPhizg9Y', 'expanded_